
nikolFrom a previous post, we have this function to import a page from moinmoin, convert it and publish:


<!-- TEASER_END -->

In [1]:
def url2rst(URL):
    import urllib
    import subprocess
    f = urllib.urlopen(URL + '?action=print')
    html = f.read()
    p = subprocess.Popen(['pandoc', '--from=html', '--to=rst'],
                         stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    return p.communicate(html)[0]

In [2]:
def convert(URL, DATE, TIME, TITLE, TAGS, doit=True):
    pagerst = url2rst(URL)
  
    TIMEDTITLE = DATE[2:] + ' ' + TITLE
    SLUG = DATE[2:] + '-' + TITLE.replace(' ', '-')
    tmplt = """\
.. title: %s %s
.. slug: %s
.. date: %s %s
.. type: text
.. tags: %s
    
""" % (DATE[2:], TITLE, SLUG, DATE, TIME, TAGS)
    
    if doit:
        import os
        print TITLE, TAGS
        cmd = 'cd ..; nikola new_post --title="%s" --tags="%s"' % (TIMEDTITLE, TAGS)
        print cmd
        os.system(cmd)
        f = file(SLUG + '.rst', 'w')
        f.write(tmplt + pagerst)
        f.close()
    else:
        print tmplt

In [3]:
URL = 'https://invibe.net/LaurentPerrinet/Presentations/10-10-20_M2_MasterSciences'
DATE, TIME = '2010-10-20', '13:36:57'
TITLE, TAGS = 'Master M2 Sciences', 'Talks, ComputationalNeuroscience, sciblog'
#convert(URL, DATE, TIME, TITLE, TAGS, doit=False)

In [4]:
URL = 'https://invibe.net/LaurentPerrinet/Presentations/10-10-27_M2_MasterSciences'
DATE, TIME = '2010-10-27', '13:36:57'
TITLE, TAGS = 'Master M2 Sciences', 'Talks, ComputationalNeuroscience, sciblog'
#convert(URL, DATE, TIME, TITLE, TAGS, doit=False)

## extracting title and tags

In [5]:
def url2raw(URL):
    import urllib
    import subprocess
    f = urllib.urlopen(URL + '?action=raw')
    raw = f.read()
    return raw

In [6]:
DATE, TIME = '2009-08-20', '13:36:57'
URL = 'https://invibe.net/LaurentPerrinet/SciBlog/' + DATE
RAW = url2raw(URL)
#print RAW

Some heuristics to extract the title (always a level 1 header):

In [7]:
#import string
for line in RAW.split('\n'):
    if len(line)>2: 
        if line[0] == '=':
            if line[0:2] == '= ': 
                TITLE = line[2:-2]
                TITLE = TITLE.strip('\r')
                print "%r, %r" % (line, TITLE)
                #for char in TITLE: print char
                print ',' in TITLE
                # while TITLE[-1]==' ': TITLE = TITLE[:-1]
                #TITLE.replace?
                #TITLE = unicode(TITLE, "utf-8")
                TITLE = TITLE.replace(',', '-')
                print "%r, %r" % (line, TITLE)
                TITLE = TITLE.replace('drafts', 'toto')
                print "%r, %r" % (line, TITLE)
import re
TITLE = re.sub(',', '-', TITLE)
#TITLE = str(TITLE)
#TITLE.replace(u",", u"-")
print type(TITLE), TITLE

'= some LaTeX tips: drafts, links, margins, pdflatex =\r', 'some LaTeX tips: drafts, links, margins, pdflatex '
True
'= some LaTeX tips: drafts, links, margins, pdflatex =\r', 'some LaTeX tips: drafts- links- margins- pdflatex '
'= some LaTeX tips: drafts, links, margins, pdflatex =\r', 'some LaTeX tips: toto- links- margins- pdflatex '
<type 'str'> some LaTeX tips: toto- links- margins- pdflatex 


In [8]:
print 'some LaTeX tips: drafts / links / margins / pdflatex'.replace('/', '-')
TITLE = 'some LaTeX tips: drafts / links / margins / pdflatex'
exclude_chars =[':', ' :', ': ', '/', '  ', '--']
for char in exclude_chars:
    TITLE = TITLE.replace(char, '-')
print TITLE

some LaTeX tips: drafts - links - margins - pdflatex
some LaTeX tips- drafts - links - margins - pdflatex


Some heuristics to extract the tags (always begins with "Tags"):

In [9]:
TAGS = []
for line in RAW.split('\n'):
    for word in line.split(' '):
        if len(word)>3: 
            if word[0:3] == 'Tag': 
                TAGS.append(word[3:])
print ', '.join(TAGS)

LaTex


Some heuristics to process the rst code:

In [10]:
pagerst = url2rst(URL)

In [11]:
#import re
pagerst_ = ''
line_old = ''
for i_line, line in enumerate(pagerst.split('\n')):
    if i_line >2 :
        if not line[:4]=='`Tag':
            if not (line_old=='' and line[:6]=='------'):
                line = line.replace('/moin_static196/', 'https://invibe.net/moin_static196/')
                pagerst_ += line + '\n'
    else:
        print 'stripping title :', line
    line_old = line
print pagerst_
#https://invibe.net/moin_static196/moniker/img/angry.png

stripping title : some LaTeX tips: drafts, links, margins, pdflatex
stripping title : =================================================
stripping title : 
checking typographic style
--------------------------

-  Lint for LaTeX :
   `http://baruch.ev-en.org/proj/chktex/ <http://baruch.ev-en.org/proj/chktex/>`__

make links
----------

-  hyperref quick reference list :

   ::

       \href{URL}{text } 
       \url{URL}
       \nolinkurl{URL}

managing margins
----------------

-  to adjust margins, use

   ::

       \usepackage[margin=2.5cm]{geometry}

   then play around with the ``2.5cm`` value until it fits.

-  tips for fitting your text in the required size : `LaTeX Tips n
   Tricks for Conference
   Paper <http://www-db.stanford.edu/~manku/latex.html>`__

citations
---------

-  If you give LaTeX \\cite{fred,joe,harry,min}, its default commands
   could give something like "[2,6,4,3]"; this looks awful. One can of
   course get the things in order by rearranging the keys in the 

wrapping-up:

In [12]:
def extract(URL):
    import re
    RAW = url2raw(URL)
    TITLE = "Unknown title"
    TAGS = []
    for line in RAW.split("\n"):
        if len(line)>2: 
            if line[0:2] == "= ": 
                TITLE = line[2:-2].strip()
                TITLE.strip()
                exclude_chars =[' / ', ' /', '/ ', '/', '  ']
                for char in exclude_chars:
                    TITLE = re.sub(char, '-', TITLE)
        for word in line.split(" "):
            if len(word)>3: 
                if word[0:3] == "Tag": 
                    TAGS.append(word[3:].strip('\r').lower())
    #if not('sciblog' in TAGS): TAGS.append('sciblog')
    TAGS = set(TAGS)
    return TITLE, ", ".join(TAGS)    

the following function includes all of the above plus a hack to remove a former version of the script that created filenames with special characters.

In [58]:
def convertSciBlog(DATE, TIME='13:36:57', doit=True):
    URL = 'https://invibe.net/LaurentPerrinet/SciBlog/' + DATE
    TITLE, TAGS = extract(URL)
    #print URL, TITLE, TAGS
    pagerst = url2rst(URL)

    pagerst_parsed = ''
    line_old = ''
    finished_title = False
    for line in pagerst.split('\n'):
        if finished_title :
            if not line[:4]=='`Tag':
                if not (line_old=='' and line[:6]=='------'):
                    line = line.replace('/moin_static196/', 'https://invibe.net/moin_static196/')
                    line = line.replace('</LaurentPerrinet/', '<https://invibe.net/LaurentPerrinet/')
                    if 'SciBlog' in line: print 'Warning ', line
                    pagerst_parsed += line + '\n'
        else:
            if line[:4]=='====': finished_title = True
            print 'stripping title :', line
        line_old = line
  
    TIMEDTITLE = DATE[2:] + ' ' + TITLE

    # The following is a hack to remove old files which had improper characters
    #SLUG_OLD = DATE[2:] + '-' + TITLE.replace(' ', '-')
    SLUG = DATE[2:] + '-' + TITLE
    exclude_chars, exclude_chars_space =[':', '"', ',', ' ', '!', '--', '--'], []
    for char in exclude_chars:
        exclude_chars_space.append(char)
        exclude_chars_space.append(' ' + char)
        exclude_chars_space.append(char + ' ')
        exclude_chars_space.append(' ' + char + ' ')
    for char in exclude_chars_space:
        SLUG = re.sub(char, '-', SLUG)

    translate = {'é':'e', 'è':'e', 'ê':'e', 'à':'a', 'á': 'a', 'É': 'E', "'": '', "\.": ''}
    for char in translate.keys():
        SLUG = re.sub(char, translate[char], SLUG)
                    
    tmplt = """\
.. title: %s %s
.. slug: %s
.. date: %s %s
.. type: text
.. tags: %s

""" % (DATE[2:], TITLE, SLUG, DATE, TIME, TAGS)
    if doit:
        import os
        #print URL, TITLE, TAGS
        print TIMEDTITLE, TAGS
        f = file(SLUG + '.rst', 'w')
        f.write(tmplt + pagerst_parsed)
        f.close()
    else:
        print tmplt 

In [59]:
DATE = '2010-10-06'
URL = 'https://invibe.net/LaurentPerrinet/SciBlog/' + DATE
TITLE, TAGS = extract(URL)
TIMEDTITLE = DATE[2:] + ' ' + TITLE
print TIMEDTITLE, TAGS

10-10-06 Émergence sciblog


In [60]:
print 'nikola new_post --title="%r" --tags="%r" ' % (DATE[2:] + ' ' + TITLE, TAGS)

nikola new_post --title="'10-10-06 \xc3\x89mergence'" --tags="'sciblog'" 


In [61]:
convertSciBlog(DATE, doit=False)

stripping title : --------------
stripping title : 
stripping title : Émergence
stripping title : =========
.. title: 10-10-06 Émergence
.. slug: 10-10-06-Emergence
.. date: 2010-10-06 13:36:57
.. type: text
.. tags: sciblog




In [62]:
convertSciBlog(DATE, doit=True)

stripping title : --------------
stripping title : 
stripping title : Émergence
stripping title : =========
10-10-06 Émergence sciblog


In [55]:
done_page_list = """\
"""
page_list = """\
SciBlog/2006-11-20
SciBlog/2006-11-30
SciBlog/2007-10-22
SciBlog/2008-08-29
SciBlog/2008-11-20
SciBlog/2008-12-05
SciBlog/2009-08-19
SciBlog/2009-08-20
SciBlog/2009-08-21
SciBlog/2009-08-23
SciBlog/2009-08-26
SciBlog/2009-08-29
SciBlog/2009-11-01
SciBlog/2009-11-15
SciBlog/2009-11-22
SciBlog/2009-11-26
SciBlog/2009-12-02
SciBlog/2009-12-06
SciBlog/2009-12-09
SciBlog/2009-12-11
SciBlog/2009-12-17
SciBlog/2009-12-19
SciBlog/2010-01-27
SciBlog/2010-02-03
SciBlog/2010-02-11
SciBlog/2010-03-15
SciBlog/2010-04-07
SciBlog/2010-04-08
SciBlog/2010-04-24
SciBlog/2010-04-25
SciBlog/2010-04-26
SciBlog/2010-04-28
SciBlog/2010-05-22
SciBlog/2010-05-23
SciBlog/2010-07-08
SciBlog/2010-08-03
SciBlog/2010-08-05
SciBlog/2010-08-07
SciBlog/2010-08-09
SciBlog/2010-08-11
SciBlog/2010-08-29
SciBlog/2010-08-30
SciBlog/2010-08-31
SciBlog/2010-09-03
SciBlog/2010-09-09
SciBlog/2010-09-27
SciBlog/2010-10-05
SciBlog/2010-10-06
SciBlog/2010-10-09
SciBlog/2010-10-27
SciBlog/2010-10-31
SciBlog/2010-11-03
SciBlog/2010-11-04
SciBlog/2010-11-07
SciBlog/2010-11-09
SciBlog/2010-11-15
SciBlog/2010-11-27
SciBlog/2010-12-08
SciBlog/2010-12-20
SciBlog/2010-12-27
SciBlog/2011-01-18
SciBlog/2011-01-19
SciBlog/2011-01-25
SciBlog/2011-01-26
SciBlog/2011-01-29
SciBlog/2011-02-09
SciBlog/2011-02-21
SciBlog/2011-02-22
SciBlog/2011-03-09
SciBlog/2011-03-10
SciBlog/2011-05-04
SciBlog/2011-07-05
SciBlog/2011-07-06
SciBlog/2011-07-07
SciBlog/2011-07-08
SciBlog/2011-07-10
SciBlog/2011-07-25
SciBlog/2012-01-07
SciBlog/2012-01-18
SciBlog/2012-02-08
SciBlog/2012-02-16
SciBlog/2012-02-17
SciBlog/2012-02-22
SciBlog/2012-03-05
SciBlog/2012-03-10
SciBlog/2012-03-11
SciBlog/2012-03-12
SciBlog/2012-03-13
SciBlog/2012-03-16
SciBlog/2012-03-21
SciBlog/2012-03-22
SciBlog/2012-03-24
SciBlog/2012-03-25
SciBlog/2012-04-05
SciBlog/2012-04-17
SciBlog/2012-04-22
SciBlog/2012-04-30
SciBlog/2012-06-12
SciBlog/2012-07-10
SciBlog/2012-07-24
SciBlog/2012-08-19
SciBlog/2012-08-27
SciBlog/2012-09-04
SciBlog/2012-09-09
SciBlog/2012-09-30
SciBlog/2012-11-07
SciBlog/2012-11-28
SciBlog/2013-01-16
SciBlog/2013-01-31
SciBlog/2013-02-02
SciBlog/2013-02-04
SciBlog/2013-03-06
SciBlog/2013-03-13
SciBlog/2013-03-23
SciBlog/2013-03-25
SciBlog/2013-05-14
SciBlog/2013-06-12
SciBlog/2013-07-08
SciBlog/2013-10-26
SciBlog/2013-11-04
SciBlog/2013-11-08
SciBlog/2013-11-09
SciBlog/2014-01-01"""

In [56]:
for page in page_list.split('\n'):
    print page.replace('SciBlog/', ''),

2006-11-20 2006-11-30 2007-10-22 2008-08-29 2008-11-20 2008-12-05 2009-08-19 2009-08-20 2009-08-21 2009-08-23 2009-08-26 2009-08-29 2009-11-01 2009-11-15 2009-11-22 2009-11-26 2009-12-02 2009-12-06 2009-12-09 2009-12-11 2009-12-17 2009-12-19 2010-01-27 2010-02-03 2010-02-11 2010-03-15 2010-04-07 2010-04-08 2010-04-24 2010-04-25 2010-04-26 2010-04-28 2010-05-22 2010-05-23 2010-07-08 2010-08-03 2010-08-05 2010-08-07 2010-08-09 2010-08-11 2010-08-29 2010-08-30 2010-08-31 2010-09-03 2010-09-09 2010-09-27 2010-10-05 2010-10-06 2010-10-09 2010-10-27 2010-10-31 2010-11-03 2010-11-04 2010-11-07 2010-11-09 2010-11-15 2010-11-27 2010-12-08 2010-12-20 2010-12-27 2011-01-18 2011-01-19 2011-01-25 2011-01-26 2011-01-29 2011-02-09 2011-02-21 2011-02-22 2011-03-09 2011-03-10 2011-05-04 2011-07-05 2011-07-06 2011-07-07 2011-07-08 2011-07-10 2011-07-25 2012-01-07 2012-01-18 2012-02-08 2012-02-16 2012-02-17 2012-02-22 2012-03-05 2012-03-10 2012-03-11 2012-03-12 2012-03-13 2012-03-16 2012-03-21 2012-03-22

In [57]:
for page in page_list.split('\n'):
    convertSciBlog(DATE=page.replace('SciBlog/', ''), doit=True)

stripping title : --------------
stripping title : 
stripping title : V1 hypercolumn Coordination Meeting, 20th - 21st Nov 2006
stripping title : =========================================================
06-11-20 V1 hypercolumn Coordination Meeting, 20th - 21st Nov 2006 facets, sciblog
stripping title : Post-doctoral Position in Computational Neuroscience: "Functional, Large-scale Models of Visual Motion Perception"
stripping title : ==================================================================================================================
06-11-30 Post-doctoral Position in Computational Neuroscience: "Functional, Large-scale Models of Visual Motion Perception" facets, sciblog
stripping title : --------------
stripping title : 
stripping title : Deliverable M9-3: Workshop for definition of a detailed version of the V1 hypercolumn model
stripping title : ===========================================================================================
Warning     PDF <https://invibe.net

In [104]:
todo_page_list = """\
SciBlog/2012-12-19
SciBlog/2012-12-12
SciBlog/2011-07-18
SciBlog/2011-07-12
SciBlog/2012-10-02
"""

clean-up of failed pages

In [122]:
import os
#print os.getcwd()
for page in todo_page_list.split('\n'):
    DATE = page.replace('SciBlog/', '')
    if len(DATE)>2:
        cmd = 'rm -fr %s*' % DATE[2:]
        print cmd
        os.system(cmd)

rm -fr 12-12-19*
rm -fr 12-12-12*
rm -fr 11-07-18*
rm -fr 11-07-12*
rm -fr 12-10-02*


### Done!